Group Assignment Four AG and DH- this notebook is to export the dfs 

In [ ]:
#Import packages
import geopandas as gpd
import pandas as pd
import plotly.express as px
from sodapy import Socrata
import folium
import matplotlib
import matplotlib.pyplot as plt

In [ ]:
#Read in EJSM df and make string of tracts to be named
df = pd.read_csv(
'Data/EJSM_Scores/EJSM_Scores (1).csv' ,
dtype={
    'Tract_1':str
})

In [ ]:
#pd.read_csv('file.csv', index_col=[0])

In [ ]:
#Add number zero leading the FIPS code for merging the data with the census tract data
df['Tract_1'] = df['Tract_1'].str.zfill(11)
df.head()

In [ ]:
df.info()

In [ ]:
#Import 2012 census data
tracts=gpd.read_file('Data/CensusData2012/census-tracts-2012.geojson')
print(tracts)

In [ ]:
#List column names 
list(tracts)

In [ ]:
#Select unwanted columns
columns_to_drop = ['set','kind','resource_uri','metadata']
#read columns 
tracts.head()

In [ ]:
#Drop unnecessary columns from tracts data
tracts = tracts.drop(columns_to_drop,axis=1)

In [ ]:
#Show tracts info
tracts.info(verbose=True, null_counts=True)

In [ ]:
#Isolate the FIPS code and geometry columns to match with the EJSM data
tracts = tracts[['name','geometry']]
tracts.head()

In [ ]:
#Show isolated columns - COULD ADD tracts.columns
tracts.columns = ['FIPS','geometry']
tracts.head()

In [ ]:
#Rename Tract_1 to FIPS
df.columns = ['OBJECTID',
 'FIPS',
 'CIscore',
 'HazScore',
 'HealthScore',
 'SVscore',
 'CCVscore',
 'Shape__Area',
 'Shape__Length']
df.head()

In [ ]:
#This section will explore the merged data

In [ ]:
#List new df columns
list(df)

In [ ]:
#Merge data on the same object FIPS
tracts_ejsm=tracts.merge(df,on="FIPS")
#Show merge with census data
tracts_ejsm.head()
tracts_ejsm.info()

In [ ]:
tracts_ejsm.info()

In [ ]:
df=tracts_ejsm

In [ ]:
df.info()

In [ ]:
#converted df to geojson 
df.to_file("tracts_ejsm.geojson", driver='GeoJSON')

In [ ]:
tracts_ejsm.crs

In [ ]:
#last time we see EJSM multipolygon data
tracts_ejsm.head()

In [ ]:
#NULL- convert csv
#df = tracts_ejsm.to_csv('Mangonadas\Data\tracts_ejsm.csv')

In [ ]:
#NULLread in csv- READ AS RACTS IN DATA FOLDER
#df = pd.read_csv(
#'Mangonadas\Data\tracts_ejsm.csv',
#dtype={
#    'Tract_1':str
#})

In [ ]:
df.info()

In [ ]:
#Describe stats by proximity to hazard score
tracts_ejsm['CIscore'].describe()

In [ ]:
#Plot histogram for cumulative impact score
tracts_ejsm['CIscore'].plot.hist(bins=50),
plt.xlabel('Cumulative Impact Scores')
plt.ylabel('Frequency'),
plt.savefig('CIscore.png')

In [ ]:
#Plot histogram for proximity to hazard score. It is grouped by the number of occurences. 
#Our not so great graph! Seeing this bar chart intrigued us to durther explore the variations between census tracts.
tracts_ejsm['HazScore'].plot.hist(bins=50)
plt.xlabel('Proximity to Hazards')
plt.ylabel('Frequecy'),
plt.savefig('HazScore_bar.png')

It’s unclear why the hazard score data is not distributed the same way as the cumulative impact score data. We wondered if it was because we had miswritten it or if the hazard occurrences were not visible on this bar graph. What is apparent is the average cumulative impact score of LA lying somewhere between 10.5-12.5. The cumulative score considers all the scores identified from the dataset included proximity to hazards, social vulnerability, climate vulnerability, and health vulnerability. 

In [ ]:
#New object to show the sorted data- these are the top polluted census tracts. The census tracts that are shown as being the most polluted are also the most socially vulnerable.
tracts_ejsm_sorted = tracts_ejsm.sort_values(by='CIscore',ascending = False)
tracts_ejsm_sorted[['FIPS','CIscore','HazScore','SVscore']].head(10)

In [ ]:
#Plot represents the proximity to environmental hazards score by equal intervals 
#We can see that the most polluted areas are in south central LA county 
tracts_ejsm.plot(figsize=(24,20),
                 column='HazScore',
                 legend=True, 
                 scheme='equal_interval')

In [ ]:
#Plot represents the cumulative impact scores separated by natural breaks
#The map shows increased variation because of the integrated scores
tracts_ejsm.plot(figsize=(24,20),
                 column='CIscore',
                 legend=True, 
                 scheme='NaturalBreaks')

This Chloropleth map provides us with a background map that will kick off the rest of our work. We chose hazard scores because while the cumulative impact was meaningful, we wanted to focus on a standard that could be more decisively measured (proximity to environmental hazards in the form of pollutants) rather than other measurements. Measurements include a social vulnerability that could be more skewed based on indicators defined by the source (in this case, USC/Occidental College). For example, although poverty and policing can be linked, we wanted to see if sites that were contaminated or close to toxic sites (such as Exide) would have a significant relationship to policing.

<div class="alert alert-danger">
There is a problem with the way you are importing the data. You have a limit of 10,000 records (this can be upped to 50,000), and you do not sort the data in any way. The resulting data is a subset of records of the total.
    
Consider instead to specify a date range and a query that makes the data fall under the 50000 record limit.

I provide an example below (df2) for your reference.
</div>

In [ ]:
#Importing our arrest 2020 data using an API
client = Socrata("data.lacity.org", None)
results = client.get("amvf-fr72", 
                     limit=50000,
                     where = "(arst_date between '2020-01-01T00:00:00' and '2020-10-30T00:00:00') AND (grp_description in ('Aggravated Assault','Miscellaneous Other Violations','Driving Under Influence','Other Assaults','Narcotic Drug Laws'))",
                     order='arst_date desc')
df =pd.DataFrame.from_records(results)

In [ ]:
df.shape

<div class="alert alert-danger">
This example limits the import to certain crime types (look at the list within the where statement). Adjust these as necessary.
</div>

In [ ]:
#Sample data to check for inconsistencies
df.sample(7)

In [ ]:
#Get description of arrest data 
df.grp_description.unique()

In [ ]:
#Examining all data types in the df for floats
df.lon = df.lon.astype('float')
df.lat = df.lat.astype('float')

In [ ]:
df.info()

<div class="alert alert-danger">
As you can see, the plot below paints a different picture.
</div>

In [ ]:
#Plotly bar graph to examine the total arrests by LAPD in 2020
px.bar(df,
       x='arst_date',
       title='LAPD Arrests in 2020',
       labels={'arst_date':'Arrest date','counts':'Number of arrests'}
      ) 

This has been a graph circulating, but it’s interesting to note how arrests boom in the summer months and the moments leading up to it due to the uprisings. What would these look like across time? When hearing about the ‘92 uprising, a familiar anecdote is how heat and power shortages aggravated communities already frustrated and fed up with the policing system. While it’s hard to test this, it’s important to note how many people, particularly folks of color, we’re stuck at home due to quarantine during summer, and many lacked air conditioning (or income to pay for utilities).

In [ ]:
#Index arrest charge value counts
arrest_by_charge = df.grp_description.value_counts().reset_index()
arrest_by_charge

<div class="alert alert-danger">
Again, these numbers will change with a more accurate import.
</div>

In [ ]:
#Index arrest charge value counts
arrest_by_charge = df.grp_description.value_counts().reset_index()
arrest_by_charge

In [ ]:
#Configure geometry for crime geodataframe
crime = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.lon, df.lat))
crime.drop(crime[crime.lon==0].index,inplace=True)

In [ ]:
#Scatterplot of crime data
crime.plot(figsize=(12,12),color='purple')

In [ ]:
#Add base from tracts data
base = tracts_ejsm.plot(figsize=(12,10),color='gainsboro', edgecolor='white')

ax = crime.plot(ax=base, color='purple', markersize=5)

In [ ]:
#Print map with within closer boundaries- IGNORE ERROR FOR NOW
base = tracts_ejsm.plot(figsize=(12,12),color='gainsboro', edgecolor='white')
ax = crime.plot(ax=base, marker='o', color='purple', markersize=5)
ax.set_xlim(minx - .1, maxx + .1)
ax.set_ylim(miny - .1, maxy + .1)
ax

In [ ]:
#Configure coordinate reference system for tracts to join the two dataframes
tracts_ejsm.crs

In [ ]:
#Configure coordinate reference system for crime
crime.set_crs(epsg=4326, inplace=True)
crime.crs

In [ ]:
#Join tracts_ejsm and crime data
join = gpd.sjoin(tracts_ejsm,
                 crime,
                 how='right')

In [ ]:
join.head()

In [ ]:
join.sample(5)

In [ ]:
#Subset the data from join
df = pd.DataFrame(join,columns=['arst_date','rpt_id','grp_description','arst_typ_cd','HazScore','CIscore','FIPS','lat','lon','geometry'])
df.info()

In [ ]:
#Subset of the joined dataframes 
#We are keeping these columns because our analysis focuses on the HazScore and description of the charges
df.head()

Here we follow Yoh’s steps more closely, but the aggravated arrests don’t tell us much. We’re trying to figure out crimes linked explicitly to environmental hazards. For now, we want to keep this large dataset because we want to be able to pick them out later depending on what crimes we want to analyze in the nexus with environmental hazards.

In [ ]:
#These are the tracts with the highest number of arrests. We included this as a priliminary step to mapping the full counts. 
crime_by_tracts = join.FIPS.value_counts().rename_axis('Tract').reset_index(name='crime_count')
crime_by_tracts.head()

In [ ]:
#This is a visualization to compare the amount of arrests in different census tracts
crime_by_tracts[:50].plot.bar(figsize=(20,8),x='Tract',y='crime_count')

This bar graph tells us the crime by census tract, but we can only guess what that is. It would be useful to bring this into the function, but we’re looking for a better way of organizing and visualizing the data. Instead of looking at specific census tracts themselves, our goal is to understand the interconnected relationship between proximity to hazards (such as pollution) and seeing what charges the police are giving people in that area. Surprisingly, the census tract with the most crimes (06037980028) is the one belonging to the airport and in between the Chevron Oil Refinery and the Ballon Wetlands


In [ ]:
#Grouped data by the description of arrest. Here, the FIPS out of order because we are descibing a full count of charge descriptions. 
df.groupby(['grp_description']).count()

In [ ]:
#Group data by the FIPS code to explore the meaning of the counts in each census tract. 
df.groupby(['FIPS']).count()

In [ ]:
#Data is grouped for counts of reports in tracts. 
df_grouped=df.groupby(['FIPS','grp_description','lat','lon','arst_date']).count()[['rpt_id']]
df_grouped.head(50)

In [ ]:
#Flatten data to prep for bar graph and reset the index to include the group values 
df_flat = df_grouped.reset_index()
df_flat

Look at how beautiful that index is. What is not shown in our flattened data frame is the amount of blood and tears (and lots of green tea caffeine) that helped us get to this point. Originally while our workflow was simple, things became more complicated as we tried to clean up our notebooks without taking away code that would serve us moving forward. This data frame allows us to see all types of crimes and arrest data across census tracts and their location. This provides us with the top-most layer for our future maps, in addition to a versatile data frame that can give temporal nuance to our maps, an attribute especially useful for the Kepler map. 

<div class="alert alert-danger">
Modified the code below (added marker_line_width=0) to get rid of the white horizontal lines that were making the bars difficult to read.
</div>

In [ ]:
#Make plotly bar graph
fig = px.bar(df_flat,
       x='grp_description',
       y='rpt_id',
       title='Description of LAPD Arrests in 2020',
       color='grp_description',
       labels={'grp_description':'Arrest Decription','rpt_id':'Number of Arrests'}
      )
fig.update_traces(marker_line_width=0)

So far, so good. It was tricky to make sense of all these numbers, but having everything cleanly indexed would provide us more than enough for what we needed. Looking back now, we would like to have examined a larger sample, but based on our gigabyte limit on Jupyter Hub, we were unsure how far to push this. From this bar graph, it becomes clear that aggravated assault, driving under the influence, and other crimes are the largest ones. We’re trying to focus on aggravated assault, disturbing the peace, and disorderly contact, because these are the ones we believe can connect policing to proximity to hazards (the ejsm score). It’s clear that aggravated assault is the largest sample; there are significantly less for disorderly conduct and disturbing the peace. For now, we don’t know if this will affect the statistical significance of our findings later on. 

In [ ]:
#Define the merged dataframe
df_ejsm = tracts_ejsm.merge(df_flat,on="FIPS")
df_ejsm.info()

In [ ]:
#Convert lat and lon to floats intergrating arrest and EJSM score data
df_ejsm['lat'] = df_ejsm['lat'].astype(float)
df_ejsm['lon'] = df_ejsm['lon'].astype(float)
df_ejsm.info()

In [ ]:
#Convert dataframe to crs
df_ejsm.crs

In [ ]:
#Create basic scatter of df_ejsm_crime_tracts data with coordinate reference system
px.scatter(df_ejsm,
           x='lon',
           y='lat'
          )

In [ ]:
#Exploring map styles using mapbox
fig = px.scatter_mapbox(df_ejsm,
                        lat='lat',
                        lon='lon',
                        mapbox_style="stamen-terrain")
fig.show()

In [ ]:
import contextily as ctx

In [ ]:
census_tracts = tracts_ejsm[['FIPS','HazScore','geometry']]
census_tracts.head()
census_tracts.info()

In [ ]:
#should this be 4326?
census_tracts = census_tracts.to_crs(epsg=3857)

In [ ]:
#Use census tracts and ejsm integrated layers to add base map
ax=census_tracts.plot(figsize=(20,20), 
                      edgecolor='white',
                      column='HazScore',
                      alpha=0.5)

ax.axis('off')

ctx.add_basemap(ax,source=ctx.providers.CartoDB.Positron)

In [ ]:
#Make crime GeoDataFrame
crime = gpd.GeoDataFrame(df_flat, 
                         crs='EPSG:4326',
                         geometry=gpd.points_from_xy(df_flat.lon, df_flat.lat))

In [ ]:
#Convert crime to crs
crime = crime.to_crs(epsg=3857)
crime.info()

In [ ]:
#Use geometry to define area
crime.geometry.total_bounds

In [ ]:
#Print total bounds
minx, miny, maxx, maxy = crime.geometry.total_bounds
print(minx)
print(maxx)
print(miny)
print(maxy)

In [ ]:
#Configure function map by census tracts as the base map
base = census_tracts.plot(figsize=(20,20), 
                      edgecolor='white',
                      column='HazScore',
                      alpha=0.5)

ax = crime.plot(ax=base, marker='o', color='purple', markersize=5)

ax.set_xlim(minx - 7000, maxx + 7000) 
ax.set_ylim(miny - 7000, maxy + 7000)

ax.axis('off')

ctx.add_basemap(ax,source=ctx.providers.CartoDB.Positron)


In [ ]:
#Define bounds for the map using the function
crime.geometry.total_bounds
minx, miny, maxx, maxy = crime.geometry.total_bounds
print(minx)
print(maxx)
print(miny)
print(maxy)

In [ ]:
def map_arrests_by_type(charge='Aggravated Assault'):
    
    crime_type = crime[crime.grp_description==charge]

    minx, miny, maxx, maxy = crime_type.geometry.total_bounds
    
    arrests_in_tracts = gpd.sjoin(census_tracts,crime_type,how='right')

    base = census_tracts.plot(figsize=(20,20), 
                    edgecolor='white',
                    column='HazScore',
                    alpha=0.5)

    ax = arrests_in_tracts.plot(ax=base, 
                                    column='grp_description',
                                    marker='o',
                                    markersize=40, 
                                    legend=True,
                                    cmap='tab20',
                                    legend_kwds={
                                       'loc': 'upper right',
                                       'bbox_to_anchor':(1.3,1)
                                    }                  # this puts the legend to the side
                                )

    ax.set_xlim(minx - 200, maxx + 200) # added/substracted value is to give some margin around total bounds
    ax.set_ylim(miny - 200, maxy + 200)

    ax.axis('off')

    ax.set_title('Occurences of '+crime.grp_description.values[0]+' in Los Angeles',fontsize=20)

    ctx.add_basemap(ax,source=ctx.providers.CartoDB.Positron)
    ax

In [ ]:
#Map arrests by using the function
map_arrests_by_type(charge='Aggravated Assault')

In [ ]:
import seaborn as sns

# to explore point patterns
from pointpats import centrography
from matplotlib.patches import Ellipse
import numpy
# import that interact library
import ipywidgets as widgets
from ipywidgets import interact, interact_manual

In [ ]:
#slice datadf
map_arrests_by_type = crime[crime.grp_description== 'Aggravated Assault'].head()

In [ ]:
#map arrests by type info- this is a subset of crime ag-- i dont think i understand this
map_arrests_by_type.info()

In [ ]:
#current df
df.info()

In [ ]:
#overloaded 
# table
#display(df[df.grp_description == 'Aggravated Assault'].head()) 

# map
#ax = df[df.grp_description == 'Aggravated Assault'].plot(figsize=(9,9), markersize=1)

# axis
#ax.axis('off')

# basemap
#ctx.add_basemap(ax,source=ctx.providers.CartoDB.DarkMatter)

In [ ]:
#ag converted- gpd points
df = gpd.GeoDataFrame(df, 
                     crs='EPSG:4326',
                     geometry=gpd.points_from_xy(df.lon, df.lat))

In [ ]:
df.info()

In [ ]:
#Change df to crs (web mercator for seaborne) crs 4326?
#df=df.to_crs('EPSG:3857')

In [ ]:
df=df.to_crs('EPSG:4326')

In [ ]:
#Make x and y variables with only aggravated assault 
df['x'] = df.geometry.x
df['y'] = df.geometry.y
df[df.grp_description == 'Aggravated Assault'].head()

In [ ]:
#data mini 
data_mini = df[df.grp_description.isin(['Aggravated Assault'])]
data_mini.head()

In [ ]:
#uses xand y from mini
sns.relplot(data=data_mini,
            x='x', 
            y='y')

In [ ]:
#Interactive seaborne map
@interact
def sns_styles(style=['darkgrid', 'whitegrid', 'dark', 'white', 'ticks']):
    # seaborn comes with themes to make them prettier
    sns.set_style(style)

    # scatterplot 
    sns.relplot(data=data_mini,
                x='x', 
                y='y')

In [ ]:
#Find centrography of map
mean_center = centrography.mean_center(df[['x','y']])
med_center = centrography.euclidean_median(df[['x','y']])

In [ ]:
#map the mean center and median center of map
sns.set_style('dark')

# Set up figure and axis
f, ax = plt.subplots(1, figsize=(9, 9))

# Plot points
ax.scatter(df['x'], df['y'], s=0.75)
ax.scatter(*mean_center, color='red', marker='x', label='Mean Center')
ax.scatter(*med_center, color='limegreen', marker='o', label='Median Center')

ax.legend()

# add a basemap
ctx.add_basemap(ax,source=ctx.providers.CartoDB.DarkMatter)
# Display
plt.show()

In [ ]:
#set orientation
major, minor, rotation = centrography.ellipse(df[['x','y']])

In [ ]:
# set the style without grid lines- 
sns.set_style('dark')

# Set up figure and axis
f, ax = plt.subplots(1, figsize=(9, 9))

# Plot points
ax.scatter(df['x'], df['y'], s=0.75)
ax.scatter(*mean_center, color='red', marker='x', label='Mean Center')
ax.scatter(*med_center, color='limegreen', marker='o', label='Median Center')

# Construct the standard ellipse using matplotlib
ellipse = Ellipse(xy=mean_center, # center the ellipse on our mean center
                  width=major*2, # centrography.ellipse only gives half the axis
                  height=minor*2, 
                  angle = numpy.rad2deg(rotation), # Angles for this are in degrees, not radians
                  facecolor='none', 
                  edgecolor='red', linestyle='--',
                  label='Std. Ellipse')

ax.add_patch(ellipse)

ax.legend()

# add a basemap
ctx.add_basemap(ax,source=ctx.providers.CartoDB.DarkMatter)

# Display
plt.show()

This map displays the concentration of arrests with the median center and mean center. The ellipse provides a general idea about the concentration of charges. However, it is better to use the median for our analysis going forward to avoid the outliers. 

In [ ]:
#clusters show all crimes in tracts where the haz score is greater than 4
sns.relplot(data=df[df['HazScore']>4],
            x='x', 
            y='y')

This plot describes areas where hazscores are above four. The concentrations are similar to the arrest plots. 

In [ ]:
#Seaborne map 
g = sns.jointplot(data = df,
                  x='x', 
                  y='y',
                  hue='grp_description',
                  s=10)

This seaborn plot shows the concentration of different charge types in Los Angeles. On the sides of the map, we can see how the concentrations are higher in areas closer to the center of the map. Instances of aggravated assualt are higher in central LA. Despite not having a basemap, this KDE plot is useful in renderin an overall view that describes the distribution of charges. 

In [ ]:
#make df to gdf from hazscore per tract estimation? to normalize data?
df.info()

In [ ]:
#this is where we had trouble with the ratio
#ratio is lacking the multiploygon framing for base map for chloropleth 
#could be a problem without the column names named
#problem is that the haz score is not by 1000s 
#STEPS
#define gdf
#figure out pop and reduce it by FIPS

#code

#gdf['g'] = gdf['arrests_per_HazScore']/gdf['TotalPop']*10


In [ ]:
#exploration with hazscore 
sns.jointplot(data=df[df['HazScore']>4],
              x='x', 
              y='y', 
              kind='kde')
ax.axis('off')

ctx.add_basemap(ax,source=ctx.providers.CartoDB.DarkMatter)

In [ ]:
#check df 
df.head()

In [ ]:
#HERE IS THE READ MISSING UNAMED

#dftec=pd.read_csv(
#'Mangonadas\Data\dftec.csv')
#print(dftec)

In [ ]:
#MADE GEOJSON WITH YOH
df.to_file("dftec.geojson", driver='GeoJSON')

## Interactive maps

In [ ]:
@interact
def charge_ellipse(charge=df.grp_description.unique().tolist()):
    # filter the data by race
    charge_filtered = df[df.grp_description == charge]

    # mean center and median
    mean_center = centrography.mean_center(charge_filtered[['x','y']])
    med_center = centrography.euclidean_median(charge_filtered[['x','y']])

    # standard ellipse
    major, minor, rotation = centrography.ellipse(charge_filtered[['x','y']])

    # Set up figure and axis
    f, ax = plt.subplots(1, figsize=(9, 9))

    # plot arrest points
    ax.scatter(charge_filtered['x'], charge_filtered['y'], s=0.75)

    # add the mean and median center points
    ax.scatter(*mean_center, color='red', marker='x', label='Mean Center')
    ax.scatter(*med_center, color='limegreen', marker='o', label='Median Center')

    # heatmap
    sns.kdeplot(x = charge_filtered.geometry.x, 
                y = charge_filtered.geometry.y,
                n_levels=20, 
                shade=False,
                thresh=0.1,
                alpha=0.3, 
                cmap='Reds', 
                ax=ax)

    # Construct the standard ellipse using matplotlib
    ellipse = Ellipse(xy=mean_center, # center the ellipse on our mean center
                      width=major*2, # centrography.ellipse db_filtered
                      height=minor*2, 
                      angle = numpy.rad2deg(rotation), # Angles for this are in degrees, not radians
                      facecolor='none', 
                      edgecolor='red', linestyle='--',
                      label='Std. Ellipse')

    ax.add_patch(ellipse)

    ax.legend()

    ax.axis('Off')

    ax.set_title(str(len(charge_filtered)) + ' incidents of crime with reported victim descent "' + crime + '"') #CHANGE HERE

    # add a basemap
    ctx.add_basemap(ax,source=ctx.providers.CartoDB.DarkMatter)
    # Display
    plt.show()

This heatmap displays the concentration of arrests with the median center and mean center. Although lacking the basemap, the image describes which areas could be more affected by the concentrations of crime.

In [ ]:
#convert df to crs 
df=df.to_crs(epsg=3857)

In [ ]:
#convert floats
df.lon = df.lon.astype('float')
df.lat = df.lat.astype('float')

In [ ]:
df.info()

In [ ]:
#plot concentrations on map with base
ax= df.plot(figsize=(12,12),
            color='red',
            markersize=1)
            
ax.axis('off')
            
ctx.add_basemap(ax,source=ctx.providers.CartoDB.Positron)

In [ ]:
#find bounds
minx, miny, maxx, maxy = df.geometry.total_bounds
print(minx)
print(maxx)
print(miny)
print(maxy)

In [ ]:
#THIS IS WHERE WE START EXPLORATION FOR OPTION 1
#we are attempting to start the spatial auto correlation for our final :)
map_arrests_by_type.info

In [ ]:
#make gdf from df
gdf = df

In [ ]:
gdf.head()

In [ ]:
#reset index to include counts
arrests_by_gdf = gdf.FIPS.value_counts().rename_axis('FIPS').reset_index(name='arrests_count')

In [ ]:
arrests_by_gdf.head()

In [ ]:
#merge arrests by FIPS to find the arrest counts to see the average in each FIPS by hazscore- this is where we got stuck with the ratio
gdf=gdf.merge(arrests_by_gdf,on='FIPS')
gdf.tail()

In [ ]:
#describe arrests per FIPS
arrests_by_gdf[:30].plot.bar(figsize=(20,4),
                             x='FIPS',
                             y='arrests_count')

In [ ]:
gdf=gdf.merge(arrests_by_gdf,on='FIPS')

In [ ]:
#could run inso problems with crs left and right not matching, but I did an override
df.crs
df.to_crs(epsg=4326, inplace=True)

In [ ]:
pointInPoly = gpd.sjoin(tracts_ejsm, df, how='right')
pointInPoly.head()

In [ ]:
#join counts- need to get counts by FIPS for ratio
crime_ct=join.FIPS_x.value_counts().rename_axis("FIPS_x").reset_index(FIPS_x="crime_count")
crime_ct.head()

In [ ]:
#need count
gdf['arrests_per_HazScore'] = gdf['arrests_count']/gdf['Hazscore']*100
gdf.sort_values(by="arrests_per_HazScore").head()

In [ ]:
#gdf of arrests by hazscore ratio
#gdf['arrests_per_HazScore'] = gdf['arrests_count']/gdf['HazScore']*100
#gdf.sort_values(by="arrests_per_HazScore").head()

In [ ]:
arrests[arrests.lon==0]

In [ ]:
#HERE IS THE READ MISSING UNAMED

#dftec=pd.read_csv(
#'Mangonadas\Data\dftec.csv')
#print(dftec)

Description of labor:
AG: fixed the sample size of data and did explorations with the heatmap and KDE plot
DH:seaborn maps and data exploration